# <u><b>0) Préambule au code</b></u>
# 

## <b><u>0.1) Ensemble des variables à modifier en début d'exécution du Notebook :</u></b>
### !!! On attire l'attention du lecteur/utilisateur sur les variables ci dessous :
### <b>- Les chemins d'accès de données :</b> Ils sont définis localement, et depuis peu de façon relative au répertoire contenant ce notebook. Le lecteur/utilisateur est libre de les modifier par rapport à son propre environnement, mais il faudra alors veiller à rendre cette modification cohérence avec le bon accès aux fichiers présents ci-dessous dans les répertoires <b>../Cache_fichiers/</b>, <b>../Figures/</b>, etc.
### <b>- Les booléens d'activation/désactivation :</b> Attention, ces booléens (selon qu'ils prennent True ou False) activent (ou désactivent) des fonctions telles que la sauvegarde en local de fichiers créés à partir de données calculées dans ce notebook, la sauvegarde en local de figures, etc.

In [1]:
# Booléen d'activation (ou non) de l'installation des modules/librairies nécessaires au bon fonctionnement de ce Notebook (cf partie 1.1) du Notebook).
installation = False

### Tout ce qui concerne la base de donnée
path_b2d = "../../B2D/"#"C:/Users/luked/Documents/Formation_Ingenieur_ML/Projets/P6/B2D/"

### Tout ce qui concerne les figures
# Sauvegarde des figures et chemin de données vers répertoire de stockage
sauvegarde_figure, path_fig = (
    True,
    "../../Figures/2_"
)

### Tout ce qui concerne le cache des fichiers (pour ne pas avoir à refaire tous les calculs systématiquement)
# Sauvegarde des figures et chemin de données vers répertoire de stockage
sauvegarde_fichiers, path_cache_input, path_cache_output = (
    True,
    "../../Cache_fichiers/1_",#"C:/Users/luked/Documents/Formation_Ingenieur_ML/Projets/P6/Cache_fichiers/1_",
    "../../Cache_fichiers/2_",#"C:/Users/luked/Documents/Formation_Ingenieur_ML/Projets/P6/Cache_fichiers/2_",
)


###### 
## <u><b>0.1) Installation - éventuelle - de librairies python :</b></u>
!!! Sautez cette étape si vous les avez déjà sur votre ordinateur, en mettant le booléen <i>installation</i> à <i>False</i>.

In [2]:
if installation :
    !pip install Pillow

###### 
## <u><b>0.2) Importations de librairies/modules/fonctions python :</b></u>

In [3]:
import os
### Quelques classiques des notebooks scientifiques
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
### Traitement de texe
from re import split
from bs4 import BeautifulSoup
### Traitement d'image
from PIL import Image
### Pickle
import pickle

In [4]:
### Keras
from tensorflow.keras import Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import image_dataset_from_directory, set_random_seed

In [5]:
### Transfert Learning
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocin_mobnet2

###### 
## <u><b>0.3) Définition de constantes personnelles :</b></u>

In [6]:
### On détermine un entier qui servira à gêler le random state au sein des différentes fonctions
rgn = 420

###### 
## <u><b>0.4) Définition de fonctions personnelles :</b></u>

In [7]:
def pickle_load(path, suffix_title):
    """Fonction qui charge un objet sauvegardé localement, grâce à des fonctions de la librairie pickle

    Arguments de cette fonction :
    suffix_title -- chaîne de caractère, titre de la sauvegarde en local, nécessite que la variable de chemin de données path_cache_output ait été définie avant"""
    # on ouvre le mode lecture de l'objet rangé dans le répertoire pointé par abs_path_cache_output
    file_pickel_rb = open(file=path+suffix_title,mode='rb')
    # chargement effectif
    objet = pickle.load(file_pickel_rb)
    # fermeture du mode lecture
    file_pickel_rb.close()
    del file_pickel_rb
    return objet

# 
# 
# <u><b>I) Importations des données nécessaires</b></u>

In [8]:
# on charge le meilleur modèle
model = load_model(filepath='./1_mobnet2_b_best.h5')

In [9]:
# On charge les races (classes de prédiction)
tab_races = os.listdir(path_b2d+'/Images')
tab_races = np.array([split(string=string, pattern='-')[1] for string in tab_races])

In [10]:
# On détecte les images mises à la racine du dossier contenant ce programme
files_path = np.array(
    [p for p in os.listdir() if p.endswith('png')|p.endswith('jpg')|p.endswith('jpeg')]
)

In [11]:
# Image chargées sous la forme d'un dataset
ds = image_dataset_from_directory(
    directory='./', 
    labels=None,
    label_mode=None,
    image_size=(224, 224), 
    shuffle=False,
    seed=rgn
)

Found 3 files belonging to 1 classes.


In [12]:
# Pre-processing
ds_prepro = ds.map(lambda x : (preprocin_mobnet2(x)))

# 
# 
# <u><b>II) Prédictions</b></u>

In [13]:
tab_pred = model.predict(ds_prepro)

1/1 [==============================] - 11s 11s/step


In [16]:
print('Titre image  ;  Race la plus probable ; Probailité de prédiction')
print('---------------------------------------------------------------')
for n, pred in enumerate(tab_pred) :
    print(files_path[n],' ', tab_races[pred.argsort()[-1]],' ', pred[pred.argsort()[-1]],'\n')

Titre image  ;  Race la plus probable ; Probailité de prédiction
---------------------------------------------------------------
image.png   African_hunting_dog   0.9999999 

labrador-retriever-1384.jpg   Labrador_retriever   1.0 

n02085620_10074.jpg   Chihuahua   1.0 



Le labrador et le chihuahua sont correctement prédits.

En revanche, Snooky (image.png), le chien en image initiale du projet, est confondu avec un chien de chasse africain. 

On peut éventuellement expliquer celà à cause du grillage en X sur cette photo. En effet, les chiens de chasses africains (cf Base de données) ont souvent des séries de taches vaguement en X, peut être que l'algo a confondu ces deux types de formes.

Les grandes oreilles des deux chiens ont peut être pu jouer également.